In [48]:
#Specify the schema for the crime data set.

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, DoubleType

schema = StructType([ 
    StructField("X",DoubleType(),True), 
    StructField("Y",DoubleType(),True), 
    StructField("RowID",IntegerType(),True), 
    StructField("CrimeDateTime", StringType(), True), 
    StructField("CrimeCode", StringType(), True), 
    StructField("Location", StringType(), True),
    StructField("Description", StringType(), True),
    StructField("Inside_Outside", StringType(), True),
    StructField("Weapon", StringType(), True),
    StructField("Post", IntegerType(), True),
    StructField("District", StringType(), True),
    StructField("Neighborhood", StringType(), True),
    StructField("Latitude", DoubleType(), True),
    StructField("Longitude", DoubleType(), True),             
    StructField("GeoLocation", StringType(), True),
    StructField("Premise", StringType(), True),
    StructField("VRIName", StringType(), True),
    StructField("Total_Incidents", IntegerType(), True),
    StructField("Shape", StringType(), True)
  ])

In [49]:
#Read the file using the schema definition

spark = pyspark.sql.SparkSession.builder.getOrCreate()

df = spark.read.csv("Part1_Crime_data.csv", sep=",", header=True, nullValue="NA", schema=schema)
df.show()

+----------------+----------------+-----+--------------------+---------+--------------------+--------------+--------------+-------+----+---------+--------------------+--------+---------+------------------+---------------+------------+---------------+-----+
|               X|               Y|RowID|       CrimeDateTime|CrimeCode|            Location|   Description|Inside_Outside| Weapon|Post| District|        Neighborhood|Latitude|Longitude|       GeoLocation|        Premise|     VRIName|Total_Incidents|Shape|
+----------------+----------------+-----+--------------------+---------+--------------------+--------------+--------------+-------+----+---------+--------------------+--------+---------+------------------+---------------+------------+---------------+-----+
|1420074.13302107|594160.602354662|    1|2022/02/26 04:00:...|       4E|   200 W MONUMENT ST|COMMON ASSAULT|          null|   null| 124|  CENTRAL|        MOUNT VERNON| 39.2975| -76.6193|(39.2975,-76.6193)|           null|        

In [50]:
#Cache the DataFrame

df1 = df.cache()


In [51]:
#Show the count of the rows

df1.count()

513158

In [52]:
# Print the schema

df1.printSchema()

root
 |-- X: double (nullable = true)
 |-- Y: double (nullable = true)
 |-- RowID: integer (nullable = true)
 |-- CrimeDateTime: string (nullable = true)
 |-- CrimeCode: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Inside_Outside: string (nullable = true)
 |-- Weapon: string (nullable = true)
 |-- Post: integer (nullable = true)
 |-- District: string (nullable = true)
 |-- Neighborhood: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- GeoLocation: string (nullable = true)
 |-- Premise: string (nullable = true)
 |-- VRIName: string (nullable = true)
 |-- Total_Incidents: integer (nullable = true)
 |-- Shape: string (nullable = true)



In [53]:
#Display first 5 rows

df1.take(5)

[Row(X=1420074.13302107, Y=594160.602354662, RowID=1, CrimeDateTime='2022/02/26 04:00:00+00', CrimeCode='4E', Location='200 W MONUMENT ST', Description='COMMON ASSAULT', Inside_Outside=None, Weapon=None, Post=124, District='CENTRAL', Neighborhood='MOUNT VERNON', Latitude=39.2975, Longitude=-76.6193, GeoLocation='(39.2975,-76.6193)', Premise=None, VRIName=None, Total_Incidents=1, Shape=None),
 Row(X=1411374.22509631, Y=589791.383964529, RowID=2, CrimeDateTime='2022/02/26 01:26:30+00', CrimeCode='9S', Location='2100 FREDERICK AVE', Description='SHOOTING', Inside_Outside='Outside', Weapon='FIREARM', Post=835, District='SOUTHWEST', Neighborhood='CARROLLTON RIDGE', Latitude=39.2856, Longitude=-76.6501, GeoLocation='(39.2856,-76.6501)', Premise='COMMON BUSINESS', VRIName='Tri-District', Total_Incidents=1, Shape=None),
 Row(X=1411401.16887136, Y=582761.775193539, RowID=3, CrimeDateTime='2022/02/26 10:22:00+00', CrimeCode='4E', Location='2000 GRINNALDS AVE', Description='COMMON ASSAULT', Insid

In [54]:
#What are distinct crime codes?
df1.select("CrimeCode").distinct().show()

+---------+
|CrimeCode|
+---------+
|       3P|
|       3K|
|      3BJ|
|       1A|
|       3M|
|       5F|
|       4B|
|       3B|
|       7A|
|      3NF|
|      3EF|
|       3N|
|       5D|
|      3LO|
|      3AF|
|       7B|
|      3GO|
|     3AJF|
|      8AO|
|       7C|
+---------+
only showing top 20 rows



In [55]:
#Count the number of crimes by the crime codes and order by the resulting counts in descending order

df1.createOrReplaceTempView('users')

query = """
SELECT CrimeCode, COUNT(*) as count
FROM users
GROUP BY CrimeCode
ORDER BY count DESC
"""
output = spark.sql(query)
output.show(10)

+---------+-----+
|CrimeCode|count|
+---------+-----+
|       4E|91650|
|       6D|68427|
|       5A|43928|
|       7A|40274|
|       6J|27636|
|       6G|26858|
|       6E|24300|
|       6C|23227|
|       4C|22438|
|       5D|14971|
+---------+-----+
only showing top 10 rows



In [56]:
#Which neighborhood had most crimes?

df1.createOrReplaceTempView('users')

query = """
SELECT Neighborhood, COUNT(*) as count
FROM users
GROUP BY Neighborhood
ORDER BY count DESC
"""
output = spark.sql(query)
output.show(1)


+------------+-----+
|Neighborhood|count|
+------------+-----+
|    DOWNTOWN|17799|
+------------+-----+
only showing top 1 row



In [62]:
#Which month of the year had most crimes?

import pyspark.sql.functions as f

df_month = df1.withColumn('month', f.month(f.to_timestamp('CrimeDateTime', 'yyyy/MM/dd hh:mm:ss+00')))\
            .withColumn('year', f.year(f.to_timestamp('CrimeDateTime', 'yyyy/MM/dd hh:mm:ss+00')))
df2 = df_month.na.drop(subset=['year', 'month'])

df2.groupBy(["year", "month"]).count().orderBy('count', ascending=False).show(1)

+----+-----+-----+
|year|month|count|
+----+-----+-----+
|2011|    7| 1753|
+----+-----+-----+
only showing top 1 row



In [58]:
#What weapons were used? 

df3 = df1.na.drop(subset=['Weapon'])
df3.select('Weapon').distinct().show()

+-------+
| Weapon|
+-------+
|  HANDS|
|  KNIFE|
|  OTHER|
|   FIRE|
|FIREARM|
+-------+



In [59]:
#Which weapon was used the most? 

df3.createOrReplaceTempView('users')

query = """
SELECT Weapon, COUNT(*) as count
FROM users
GROUP BY Weapon
ORDER BY count DESC
"""
output = spark.sql(query)
output.show(1)

+-------+-----+
| Weapon|count|
+-------+-----+
|FIREARM|46139|
+-------+-----+
only showing top 1 row

